*Le travail présent a été réalisé par une étudiante dans le cadre du cours "Agriculture en environnement controlé" donné en Master 2 à Gembloux Agro-bio Tech (Belgique) par le professeur Fédéric Lebeau. Ce travail a été également encadré par Arnaud Bouvry, assistant-doctorant.*

#                                     Ray-tracing des rayons dans une chambre de culture

###                                                               Marie Picron   

##### 1.Introduction 

La croissance démographique entrainant une pression sur la suffisance alimentaire et l'agriculure menacée par les conditions métérologiques et par la disponibilité des ressources, il est nécessaire aujourd'hui de repenser à une nouvelle forme d'agriculture. 

Les *Plant Factories with Artificial Lamps* (PFALs) connaissent un essor depuis quelques décinies. Ces infrastructures constituent à verticaliser les cultures et à suivre et controler l'environnement et ses divers paramètres climatiques, les entrées et les sorties. En plus d'approvisonner en alimentation, les PFALs permettent aussi de faciliter la recherche et l'éducation. En effet, des projets immergent grâce à la capacité d'intéragir et de manipuler les différents paramètres sans dépendre d'autres variables. Ainsi, l'étude et la compréhension des différents phénomènes sont de plus en plus précis et complet. 

A l'aube de l'industrie 4.0 où on tend vers un monde digitalisé, les PFALs peuvent être virtualisés pour, non seulement éviter les différentes expérimentations et manipulations chronophages et épuisant des ressources, mais aussi pour modéliser et comprendre les différents phénomènes.  

Ainsi, grâce à la virtualisation, le ray-tracing des rayons venant d'une lampe peut être modélisé afin d'observer les trajectoires de ces différents rayons et leurs intersections avec les plantes dans une chambre de culture. Cela permettra de conclure quelles sont les longueurs d'onde que la plante reçoit et comment la plante va se comporter. Aussi, les études sur l'influence des propriètes luminueux des lampes dans la chambre de culture peuvent être réalisées. 
Divers logiciels existent pour permettre le ray-tracing (donner des exemples). Le travail présent utilise le logiciel _Visualization ToolKit_ (VTK).

- ***Le logiciel VTK***

Ce logiciel open source et puissant permet de réaliser différents algorithmes dans un espace de 3 dimensions (comme la visualisation de données volumiques ou le traitement d'image). 

VTK permet de manipuler et d'intéragir facilement avec les objets sans se soucier des coordonnées spatiales et des différentes propriétes géométriques. En effet, par exemple, des algorithmes de VTK peuvent sélectionner automatiquement le centre d'une forme géométrique. Cela peut s'averer très utile pour les formes très complexes. 

Bien que ce séduisant logiciel présente de nombreux avantages, il contient, cependant, aussi de nombreux inconvenients. En effet, le site de documentation de VTK (https://vtk.org) est, personnellement, mal instruit. Un utilisateur débutant avec VTK aura plus de difficulté à prendre en main ce module. 

De plus, pour réaliser une simple opération, il est parfois nécessaire de coder plusieurs lignes et fonctions afin d'obtenir l'objectif. Il existe des architectures d'hiérachie de classes (fonctions) à procéder et à respecter avec éventuellement des options à ajouter. Avec ces différentes classes, il faut associer entre elles pour obtenir le résultat souhaité. Par exemple, pour visualiser une forme géométrique, il faut suivre la procédure suivante: 

$$
Source \xrightarrow{\text{}} Mapper \xrightarrow{\text{}} Actor \xrightarrow{\text{}} Renderer
$$

Ces différentes classes seront expliquées ultérieurement.
                                                  

### 2. Rappels Théoriques

#### Lois de Réflection

Les lois de la réflexion seront utilisées dans le cas du travail présent et sont rappelées dans ce paragraphe.
Lorsqu'un rayon lumineux (rayon incident) arrive sur une surface (point d'incidence), ce rayon peut être réfléchi de différentes manières: soit de façon spéculaire, soit de façon diffuse. Cette réflection dépend de la rugosité de la surface.

<img src ="http://localhost:8888/files/Documents/image/lo_speculardiffuse.jpg?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="300">

***Réflection Spéculaire***

La réflexion spéculaire est un phénomène où le rayon se comporte parfaitement comme un miroir et suit les lois de Snell-Descartes. Le rayon incident forme un angle avec la normale du point incident, appelé angle d'incident $\theta_i$. De la même manière, l'angle formé entre le rayon réfléchi et la normale est l'angle réfléchi $\theta_r$. La loi de Snell-Descartes démontre que les valeurs d'angles incident et réfléchi doivent être identique comme présenté sur la figure suivante. 


<img src ="http://localhost:8888/files/Documents/image/snell_descartes.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="200">

***Réfléction Diffuse***

La réflexion diffuse, quant à elle, signifie que la surface réfléchie le rayon incident aléatoirement dans toutes les directions du point d'incident. A l'inverse de la surface spéculaire où elle est parfaitement polie, la surface diffuse est quant à elle rugueuse. 

C'est deux modes de réflection évoqués sont deux cas extrèmes et irréaliste. En effet, une surface n'est pas parfaitement diffuse, ni spéculaire, mais un compromis entre les deux.  

La lumière est un peu dispersée autour du vecteur de la réflection spéculaire et forme un lobe bleu comme démontre la figure suivante. 
La forme du lobe est fonction de la rugosité de la surface. Plus une surface est rugeuse, plus le lobe sera dispersé autour de l'axe de réflection spéculaire. A l'inverse, plus une surface sera poli, plus le lobe sera fin et ressemblera à la réflexion spéculaire parfaite (comme dans le cas d'un miroir). 

Si nous émettons l'hypothèse que la surface est de nature isotrophe, le forme du lobe sera symétrique autour de l'axe de réflection spéculaire. En effet, tout déplacement d'un certain angle $\Omega$ de l'axe entraine une diminution de la luminance. Si, en gardant ce même angle d'écart $\Omega$, on se déplace autour de l'axe, la luminance sera identique. 

La diffusion peut être modélisée comme la diffusion lambertienne où la lumière est éparpillée également dans toutes les directions de la surface. Cette diffusion est représentée par le demi-cercle vert. 

<img src ="http://localhost:8888/files/Documents/image/reflection.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="300">


#### Modèle d'illumination de Phong

Pour modèliser l'approximation de la distribution angulaire du rayonnement pour chaque point incident, une approche classique est le modèle de Phong. 

Pour ce modèle, la lumière réfléchie est divisée en 3 composantes :

- **La composante ambiante** représente les parasites d'origine étrangère que la source lumineuse (par exemple la lumière réfléchie d'un autre point).

- **La composante diffuse** 

- **La composante spéculaire**

<img src ="http://localhost:8888/files/Documents/image/phong.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="400">

Pour chaque matériau, on défini des constantes caractéristiques : 
- $ka$ $\in$ [0:1] liée à la composante ambiante
- $kd$ $\in$ [0:1] liée à la composante diffuse
- $ks$ $\in$ [0:1] liée à la composante spéculaire
- $ \alpha$ liée à la brillance du matériaux

L'intensité réfléchie de la composante ambiante est donnée par $Ia =ia.ka$

L'intensité réfléchie de la composante diffuse est donnée par  $Id$ $=$ $id$.$kd$.$cos$$\theta$

L'intensité réfléchie de la composante spéculaire est donnée par $Is = is.ks.$$\cos$$^\alpha$$\Omega$

Avec $ia$ , $id$ et $is$ l'intensité incident respectivement ambiante, diffuse et spéculaire. 

L'intensité totale est la somme des trois intensité $I = Ia + Id +Is$

#### Coefficient de Fresnel et loi de Snell 

L'avenir d'un photon arrivé à un point d'incidence entre deux milieux distincts est soit d'être réfléchi, soit d'être transmis dans le nouveau milieu. 

<img src ="http://localhost:8889/files/Documents/image/reflection_transmission.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="250">


La loi de Snell permet de déterminer l'ange entre la normale et le rayon transmis, appelé l'angle de transmission ($\theta_t$) selon l'équation suivante:
$$
\eta_2sin\theta_t = \eta_1sin\theta_i   \iff  \theta_t=arcsin\left(\frac {\eta_1} {\eta_2}sin(\theta_i)\right)     
$$
Cet angle varie en fonction de l'angle d'incidence et des indices réfractives des milieux.
Il est intéressant de souligner que l'angle de transmission est limité à 90° de la normal du point incident. Au-délas de 90°, le vecteur de transmission est considéré comme un vecteur de réflection. Dans ce cas, la réflection est totale , il n'y a pas de réfraction possible. Ce phénomène se produit lorque l'angle incident est supérieur à une certaine valeur d'angle, appelée angle critique ($\theta_c$). 
$$
\theta_c=arcsin\left(\frac {\eta_2} {\eta_1}sin(90)\right) 
$$

Les probabilités qu'un rayon soit réfléchi ou transmis se calculent selon les équations de Fresnel. Ces équations dépendent de l'indice réfractive des deux milieux ($\eta_1$et $\eta_2$) ainsi de l'angle d'incidence ($\theta_i$).

La probabilité qu'un photon soit réfléchi est formulée comme la moyenne de la composante horizontale et verticale du vecteur réfléchi.
Dans le cas où il y a une réflection totale, la probabilité vaut naturellement 1.

$$
R(\theta_i) = \begin{cases}\frac{R_\perp(\theta_i) + R_\lVert(\theta_i)}{2} & \text{if not total reflection} \\1 & \text{if total reflection}\end{cases}
$$

Les composantes verticale et normales du vecteur réfléchi sont calculé selon les équations suivantes:

$$
R_\perp(\theta_i) = \left(\frac {\eta_1cos(\theta_i) - \eta_2cos(\theta_t) } {\eta_1cos(\theta_i) + \eta_2cos(\theta_t)}\right)^2
$$


$$
R_\lVert(\theta_i) = \left(\frac {\eta_2cos(\theta_i) - \eta_1cos(\theta_t) } {\eta_1cos(\theta_i) + \eta_2cos(\theta_t)}\right)^2
$$


avec $$ cos(\theta_t) = \sqrt{1-\left(\frac {\eta_1} {\eta_2}\right)^2 sin(\theta_i) ^2}
$$

La probabilité qu'un photon soit transmis est déterminée par la différence entre l'unité et la probailité de réflection.

$$
T(\theta_i) = 1 - R(\theta_i)
$$

### 3. Code 

Ce chapitre est consacré à la méthodologie de la modélisation du ray-tracing en un instant $t$ dans une chambre de culture avec trois lampes grâce au logiciel VTK couplé avec Python. 

Le ray-tracing est le **processus** du programme. Un **pré-processus** peut être effectué contenant les propriètes de la lampe. Celui-ci n'est pas développé dans le cas présent mais un ébauche est proposé dans le fichier XXXX.  
Un **post-processus** peut être aussi réalisé rassemblant les informations des rayons intersectés à la plante.

- Importation des différents Packages et du script "définition" contenant tous les fonctions utilisées. 

Le script *définition.pynb* contient les différentes fonctions nécessaires et les commentaires associés.

In [92]:
import json
import sys
import vtk
import numpy
import random
from definition import *
from datetime import datetime

- Fonctions qui permettent de convertir list/tuple en numpy et vice-versa. 

Les deux fonctions lamba permettent de convertir une liste ou un tuple en un tableau numérique et vice-versa. 
En effet, VTK utilise et génére des données telles que les coordonnées ou les vecteurs en object de type 'liste' ou 'tuple' qu'il faut convertir en objet de type 'numpy' pour effectuer des opérations vectoriels.

In [93]:
l2n = lambda l: numpy.array(l)
n2l = lambda n: list(n)

* Création du dictionnaire contenant les résultats
 & Début du chrono de l'exécution

Un dictionnaire _'plant_result'_ est créé. Il stockera les informations pour le post-processing. On enregistre aussi l'heure actuelle pour calculer la durée de l'excécution du programme. 

In [94]:
plant_result = {}
startTime = datetime.now()
print(startTime)

2020-12-28 18:07:49.674164


- Lecture fichier d'entrée .json

Le fichier d'entrée est de type json. Il fonctionne comme un dictionnaire mais permet d'être séparé du code. Cela est utile pour utiliser n'importe quel fichier de même structure dans un code. Les données des lampes et les informations de la chambre de cultures sont inscrites dans ce fichier. 

In [95]:
try:
   with open('donnee_entree.json') as f:
     data = json.load(f)
except:
   sys.exit("Error: enable to read file")

- Création de l'environnement 3D visuel

La fonction _vtkRenderer_ permet de créer l'environnement visuel où les différents objets seront ajoutés et le processus sera représenté. La couleur blanche pour le fond est choisie. 

In [96]:
ren = vtk.vtkRenderer()
ren.SetBackground(1, 1, 1) #(R,G,B)

- Création de la chambre de culture

La chambre de culture en forme de cube est conçue avec les dimensions indiquées dans le dictionnaire d'entrée Json. Il est important de souligner que l'origine des axes de ces dimensions se trouve au centre de gravité du polygone. Ainsi, par exemple pour une hauteur de 30 cm, nous aurons -15 cm et +15 cm dans l'axe Y. 

<img src ="http://localhost:8889/files/Documents/image/axes_chambre.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="350">

In [97]:
xwidth = data["room"]["width"]
ywidth = data["room"]["height"]
zwidth = data["room"]["ceiling-height"]
cube = vtk.vtkCubeSource()
cube.SetXLength(xwidth)
cube.SetYLength(ywidth)
cube.SetZLength(zwidth)
cube.Update()

_vtkPolyDataMapper_

Pour pouvoir visualiser, la géométrie est transformée en primitives graphiques.

In [98]:
cubeMapper = vtk.vtkPolyDataMapper()
cubeMapper.SetInputConnection(cube.GetOutputPort())

_Acteur_

La fonction _actor_ représente l'entitée géométrique. La forme cube est officiellement créée grâce à la combinaison des deux fonctions _Mapper_ et _Actor_.
Des options comme définir sa position, sa couleur, son opacité, etc. sont possibles à ajouter.

In [99]:
cubeActor = vtk.vtkActor()
cubeActor.SetMapper(cubeMapper)
cubeActor.SetOrigin(0, 0, 0) 
cubeActor.GetProperty().SetOpacity(0.1)
cubeActor.GetProperty().SetColor(0, 0, 1)
cubeActor.GetProperty().EdgeVisibilityOn()  # show edges
cubeActor.GetProperty().SetEdgeColor(0, 1, 0)

L'acteur est ajouté à l'environnement virtuel _ren_ prédéfini auraparavent.

In [100]:
ren.AddActor(cubeActor)

_OBBTree_

La fonction *OBBTree* est une des plus importantes du code. Il permet d'indiquer que les surfaces d'un objet sont des surfaces intersectives. Autrement dit, cette fonction spécifie que ces surfaces sont les obstacles du ray-tracing.

In [101]:
obbSurface = vtk.vtkOBBTree()
obbSurface.SetDataSet(cube.GetOutput())
obbSurface.BuildLocator()

On extrait les normales des surfaces grâce à la fonction *cellcenter_normal* dans le script *definition*. Ces normales seront utilisées pour les lois des réflections.

In [102]:
pointsCellCentersSurface, normalsSurface = cellcenter_normal(cube)

- Création de la plante

Pour simplifier dans un premier temps, une surface plane mime une plante auxquels certains rayons vont l'intersecter. La méthode est similaire que celle de la création de la chambre de culture. L'origine est le même que celle de la chambre. 

In [103]:
plant = vtk.vtkPlaneSource()
plant.SetOrigin(-30, -75, -10)  #premier point
plant.SetPoint1(-30, -75, 30)   #deuxième point
plant.SetPoint2(40, -75, -10)   #troisième point
plant.SetXResolution(15)
plant.SetYResolution(15)
plant.Update()

plantMapper = vtk.vtkPolyDataMapper()
plantMapper.SetInputConnection(plant.GetOutputPort())

plantActor = vtk.vtkActor()
plantActor.SetMapper(plantMapper)
plantActor.GetProperty().SetColor(0, 1, 0)
plantActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
plantActor.GetProperty().SetEdgeColor(0, 1, 0)

ren.AddActor(plantActor)

pointsCellCentersplante,normalsplante =cellcenter_normal(plant)

On indique que cette surface plane est une surface intersective grâce à la fonction *ObbTree*. 

In [104]:
obbPlant = vtk.vtkOBBTree()
obbPlant.SetDataSet(plant.GetOutput())
obbPlant.BuildLocator()

- Création des lampes

La suite du code est une boucle itérative où chaque lampe est calculée individuellement. En effet, pour chaque lampe, la forme sera créée et s'en suit le processus du ray-tracing de la lampe.
Pour des raisons de clareté, la boucle est scindée en plusieurs paragraphes pour pouvoir de manière plus explicite, commenter les étapes. Le code de la boucle complet se trouve à la fin de ces explications et peut être exécuté.

En fonction de son angle d'ouverture, la lampe est représentée comme une portion de la sphére. Dans le cas présent, les lampes ont un angle d'ouverture de 180°, la forme sera une demi-sphére. La méthode est similaire aux deux objets précédents. Il est important de rappeler que l'origine est située au centre du cube. La localisation de la lampe est référencée à cette origine, comme la localisation de la plante.

La résolution de la lampe est intéressante car, en plus d'arrondir au mieux la demi-sphére, le maillage de la lampe va permettre le lancer des rayons. C'est à partir des points centraux des cellules du maillage que les rayons démarrent leurs trajectoires du ray-tracing. Plus la résolution est grande, plus le nombre de cellule du maillage est élevé, plus le nombre de rayons lancés est importante.

La figure suivante montre un aperçu de l'environnement crée avec la plante et les trois lampes. 

<img src ="http://localhost:8889/files/Documents/image/axes.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="350">

*Points des cellules et normales des lampes*

Les coordonnées des points pour chaque cellule du maillage de la lampe sont calculées et le point zero qui correspond au centre de la cellule est extrait. On extrait également les normales des lampes.

- Ray-tracing

L'étape suivante du code démarre le processus du ray-tracing et est une boucle. En effet, chaque rayon et sa trajectoire, un à la fois, sont créés pour chaque lampe.

Pour chaque points centraux du maillage de la lampe, ce point et la normale associée sont extraits en variable. 

Les coordonnées du point terminal du rayon est calculé en prolongeant la normale d'un facteur de 100 à partir du point central de la cellule étudiée. Le choix du facteur 100 est justifié par la certitude que le rayon soit suffisamment long pour atteindre les parois ou la plante. 

Grâce aux fonctions préalablement définies dans le fichier *définitions*, la fonction *IntersectWithLine* qui est couplée avec la fonction *obbTree* permettent de détecter les intersections des rayons aux surfaces de la parois et à la plante.

*Intersection avec la plante*

Si une intersection avec la plante est trouvée, on extrait le numéro du point de la cellule (point incident) et ses coordonnées. Le rayon est dessiné pour la visualisation.

La normale de ce point est prélevée et le vecteur incident est mesuré. La variable X permet de déduire si le rayon est réfléchi ou transmis selon les probablités détermininées selon les coefficients de Fresnel. 

Si le rayon est transmis dans la plante, par conséquent l'énergie lumineuse est absorbée dans la matière, il est intéressant de récolter divers données pour le post-processing. Dans le cas contraire, le rayon est réfléchi et donc aucune énergie lumineuse est absorbée dans la plante.

Des informations comme les coordonnées *ptsInter*, le numéro de la lampe correspondant *v* ou encore le rayonnement sont enregistrées dans le dictionnaire *plant_result* créé au préalable. On crée une clé avec la numéro de la cellule intersectée. Si ce numéro a déjà été enregistré dans le dictionnaire, on enregistre les informations du rayon intersecté *i* dans la clé. Si ce numéro n'existait pas encore, on l'ajoute dans le dictionnaire.

*Intersection avec la parois*

Si une intersection avec la parois est trouvée, on extrait le numéro du point de la cellule (point incident) et ses coordonnées. Le rayon est dessiné pour la visualisation. 

La normale de ce point est prélevée et le vecteur incident est mesuré. La variable X permet de déduire si le rayon est réfléchi ou transmis selon les probablités détermininées selon les coefficients de Fresnel.  

Si le rayon est réfléchi, la direction de celui-ci est choisi aléatoirement. En effet, le point d'incidence est réprésenté par une sphère. Un point central d'une cellule du maillage de cette sphère est tiré aléatoirement et la normale de ce point sera la vecteur réfléchi. Un nouveau point terminal de ce rayon réfléchi est déterminé. 

Dans le cas contraire, si le rayon est transmis dans la paroi, par conséquent l'énergie lumineuse est absorbée dans la matière, il n'y a pas de réflection de la lumière dans la chambre.

Mettons en évidence la faiblesse de cette partie du code. En effet, le point incident est réprésenté par une sphère et la normale choisi aléatoirement peut être une qui est dans le milieu de transmission. Dans ce cas, nous n'avons pas un rayon réfléchi. Le problème peut être résolu en schématisant le point incident comme une demi-sphère. Cependant, l'orientation de la demi-sphère varie selon la paroi de la chambre de culture. A ce jour, une méthode pour éviter de produire un rayon réfléchi dans le milieu de transmission n'a pas encore été trouvée. 

*Réflection*

Il est possible de détecter l'intersection du rayon réfléchi avec soit la plante soit la parois. La méthode est similaire que le premier lancer d'un rayon de la lampe. 

Il est intéressant de réfléchir à une boucle pour effectuer plusieurs réfléctions.
Il est à nuancer aussi avec la limite de réfléxion où la quantité de lumière devient négligeable. 

*Le code complet de la boucle*

In [105]:
v = 0
i = 1
while v < 3:
 x = data["array"][str(v)]["x"] #par rapport à l'origine qui est au centre du cube
 y = data["array"][str(v)]["y"]
 z = data["array"][str(v)]["z"]

 Lamp = vtk.vtkSphereSource()
 Lamp.SetCenter(x, y, z)
 Lamp.SetRadius(5)
 angle_ouverture = data["array"][str(v)]["angle"]
 angle = 180 - angle_ouverture
 Lamp.SetStartTheta(180 + angle/2)
 Lamp.SetEndTheta(360 - angle/2)
 Lamp.SetPhiResolution(20)
 Lamp.SetThetaResolution(20)

 mapper = vtk.vtkPolyDataMapper()
 mapper.SetInputConnection(Lamp.GetOutputPort())

 pointActor = vtk.vtkActor()
 pointActor.SetMapper(mapper)
 pointActor.SetOrigin(0, 0, 0) 
 pointActor.GetProperty().SetColor(color(data["array"][str(v)]["wavelength"]))
 pointActor.GetProperty().EdgeVisibilityOn()  # show edges/wireframe
 pointActor.GetProperty().SetEdgeColor(color(data["array"][str(v)]["wavelength"]))

 ren.AddActor(pointActor)

 pointsCellCentersLamp,normalsLamp =cellcenter_normal(Lamp)

# #-----------------------------------------------------------------------------
# # RAY-TRACING
# #----------------------------------------------------------------------------

 for idx in range(pointsCellCentersLamp.GetNumberOfPoints()):

    pointLamp = pointsCellCentersLamp.GetPoint(idx)
    normalLamp = normalsLamp.GetTuple(idx)
    
    pointRayTarget = n2l(l2n(pointLamp) + 500*l2n(normalLamp))           
  
    if isHit(obbPlant, pointLamp, pointRayTarget):  
        ptsInter, cIdsInter = GetIntersect(obbPlant, pointLamp, pointRayTarget)
        
        normalplante = normalsplante.GetTuple(cIdsInter[0])
        vecInc = n2l(l2n(ptsInter[0]) - l2n(pointLamp))
        
        X = Fresnel(data["room"]["n_air"],data["room"]["n_plante"],vecInc, normalplante)
        if X[0] == "T" :  
             addLine(ren, pointLamp, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
             addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))
            
             key = "surface_"+str(cIdsInter[0])
          
             if key in plant_result: 
                plant_result["surface_"+str(cIdsInter[0])]['coord'][i] = ptsInter[0]
                plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
                i+=1
                 
             else:
                plant_result.update({"surface_"+str(cIdsInter[0]):{"coord":{},"lampe":{}}})
                plant_result["surface_"+str(cIdsInter[0])]['coord'][i]= ptsInter[0]
                plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
                i+=1
    
    if isHit(obbSurface, pointLamp, pointRayTarget): 
 
        pointsInter, cellIdsInter = GetIntersect(obbSurface, pointLamp, pointRayTarget)
        
        normalSurface = normalsSurface.GetTuple(cellIdsInter[0])
        vecInc = n2l(l2n(pointsInter[0]) - l2n(pointLamp))
        #vecRef = calcVecR(vecInc, normalSurface)
 
        X =Fresnel(data["room"]["n_air"],data["room"]["n_paroi"],vecInc, normalSurface)
        if X[0] == "R":
            addLine(ren, pointLamp, pointsInter[0], color(data["array"][str(v)]["wavelength"]), opacity = 0.1)
            point = addPoint(ren, pointsInter[0], color(data["array"][str(v)]["wavelength"]), opacity = 0.1)
            pointsCellCenterspoint, normalspoint =cellcenter_normal(point)
            m = random.choice(range(pointsCellCenterspoint.GetNumberOfPoints()))
            vecRef = normalspoint.GetTuple(m)
            
            pointRayReflectedTarget = n2l(l2n(pointsInter[0]) + 100*l2n(vecRef))
                
    try :          
      if isHit(obbSurface,  pointsInter[0], pointRayReflectedTarget): 
          
            pointsInter, cellIdsInter = GetIntersect(obbSurface,  pointsInter[0], pointRayReflectedTarget)
     
            addLine(ren, pointsInter[0], pointsInter[1], color(data["array"][str(v)]["wavelength"]),opacity=0.1)
            addPoint(ren, pointsInter[1], color(data["array"][str(v)]["wavelength"]), opacity =0.1)
         
      if isHit(obbPlant, pointsInter[0], pointRayReflectedTarget):  
            ptsInter, cIdsInter = GetIntersect(obbPlant,  pointsInter[0], pointRayReflectedTarget) 
            
            normalplante = normalsplante.GetTuple(cIdsInter[0])
            vecInc = n2l(l2n(pointRayReflectedTarget) - l2n(ptsInter[0]))
        
            X =Fresnel(data["room"]["n_air"],data["room"]["n_plante"],vecInc, normalplante)
        
            if X[0] =="T": 
                
                addLine(ren, pointsInter[0], ptsInter[0], color(data["array"][str(v)]["wavelength"]))
                addPoint(ren, ptsInter[0], color(data["array"][str(v)]["wavelength"]))  
                
                key = "surface_"+str(cIdsInter[0])
        
                if key in plant_result: 
                  plant_result["surface_"+str(cIdsInter[0])]['coord'][i] = ptsInter[0]
                  plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
                  i+=1
    
                else:
                 plant_result.update({"surface_"+str(cIdsInter[0]):{"coord":{},"lampe":{}}})
                 plant_result["surface_"+str(cIdsInter[0])]['coord'][i]= ptsInter[0]
                 plant_result["surface_"+str(cIdsInter[0])]['lampe'][i] = v
                 i+= 1
     
    except:
       a = 0

 v+=1 

- Résultat

*Nombres de rayons*

In [106]:
print(str(i-1)+" rayons ont atteint la plante" )
print("Chaque lampe a émis "+str(idx)+" rayons")

16 rayons ont atteint la plante
Chaque lampe a émis 719 rayons


*Durée de l'exécution*

Grâce au temps enregistré au début du code et à la fin, la durée de l'exécution est obtenue.

In [107]:
endTime = datetime.now()
deltaTime = endTime - startTime
print("Completed in "+str(deltaTime))

Completed in 0:00:03.051274


_Comparaison des temps d'exécution en fonction du nombres de rayons._

|Résolution de la lampe|Nombres de rayons|Temps d'exécution sur JupyterLab| Temps d'exécution sur Python (local)|
|----------------------|-----------------|--------------------------------|-------------------------------------|
|         10           |       159       |         0:00:02.919645         |            0:00:00.712960           |
|         20           |       719       |         0:00:06.680457         |            0:00:02.941227           |
|         30           |       1679      |         0:00:23.673360         |            0:00:15.157795
|         50           |       4799      |         0:00:25.980488         | 
|         100          |       1679      |         0:00:06.263835         | 

*Fichier de sortie*

Les résultats du ray-tracing sont obtenus dans le dictionnaire _plant_result_. Ce dictionnaire est converti en un fichier externe .json afin de pouvoir l'utiliser pour un éventuel post-processing plus avancé. 

In [108]:
with open("out_lamp.json", "w") as f:
        json.dump(plant_result, f,indent= 8)

*Visualisation*

Il est possible de visualiser la scéne et le processus du ray-tracing. Malheureusement, il est seulement possible d'exécuter ces lignes de code sur un ordinateur local (et non sur NoteBook). Néanmoins, la figure suivante montre un aperçu du résultat.

<img src ="http://localhost:8889/files/Documents/image/exemple_ray_tracing.PNG?_xsrf=2%7Cbac72762%7C792301d8141d3aaeb662b9cc90fa357e%7C1606985463" width="500">

In [109]:
#renWin = vtk.vtkRenderWindow()
#renWin.AddRenderer(ren)

#iren = vtk.vtkRenderWindowInteractor()
#iren.SetRenderWindow(renWin)

#iren.Initialize()
#renWin.Render()
#iren.Start()

### 4. Conclusions et perspectives

- indice qui dépendent de la longueur d'onde